In [2]:
import tensorflow as tf

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
import matplotlib.pyplot as plt
plt.imshow(mnist.train.images[1].reshape(28,28))

In [5]:
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784 

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1 # Stride is the shifting factor of the filter that we apply on a group of pixels
stride_conv2 = 1
conv1_k = 5 # Filter1 dimensions
conv2_k = 5 # Filter2 dimensions
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) * n_conv2 

In [6]:
# Parameters to be trained

weights = {
    # Weights that need to be trained for the first convolutional layer
    'wc1' : tf.Variable(tf.random_normal([conv1_k,conv1_k,input_channels,n_conv1])),
    # K * K * C is the dimension of the filter and K*K*C*n_conv1 is the number of parameters that we need to train for first convolutional layer
    # Weights that need to be trained for the second convolutional layer
    # We need not train any weights for the max pooling layer as it just reduce the size of the image and nothing else 
    'wc2' : tf.Variable(tf.random_normal([conv2_k,conv2_k,n_conv1,n_conv2])),
    'wh1' : tf.Variable(tf.random_normal([input_size_to_hidden,n_hidden])),
    'wo' : tf.Variable(tf.random_normal([n_hidden,n_out]))
}

biases = {
    'bc1' : tf.Variable(tf.random_normal([n_conv1])),
    'bc2' : tf.Variable(tf.random_normal([n_conv2])),
    'bh1' : tf.Variable(tf.random_normal([n_hidden])),
    'bo' : tf.Variable(tf.random_normal([n_out]))
}

In [7]:
def cnn(x,weights,biases,kprob):
    # Reshape the input from 784 to 28*28*1
    x = tf.reshape(x,shape=[-1,input_height,input_width,input_channels]) # -1 because it can infer the 4th argument itself
    conv1 = conv(x,weights['wc1'],biases['bc1'],stride_conv1)
    conv1_pool = maxpooling(conv1,max_pool1_k)
    
    conv2 = conv(conv1_pool,weights['wc2'],biases['bc2'],stride_conv2)
    conv2_pool = maxpooling(conv2,max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool,[-1,input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input,weights['wh1']),biases['bh1'])
    # hidden_output = tf.nn.relu(hidden_output_before_activation)
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout,keep_prob=kprob)
    # While training keep_prob is <1 (eg 0.8) but while testing the keep_prob is 1
    
    
    output = tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output # We haven't applied any activation function to the output layer

In [9]:
def conv(x,weights,bias,strides = 1):
    out = tf.nn.conv2d(x,weights,padding="SAME",strides=[1,strides,strides,1]) # n_ex,height,width,depth(num_of_channels)
    # strides = [] (some list) , this list should be of same shape as that of x passed
    # These 4 numbers inside strides signify how we want to apply our filter once applied
    # We are taking 1 image by 1 so we are taking first argument to be 1
    out = tf.nn.bias_add(out,bias) # bias_add as it lets you add different dimension 
    out = tf.nn.relu(out)
    return out

def maxpooling(x,k = 2): # k is the max pooling window size
    return tf.nn.max_pool(x,padding="SAME",ksize=[1,k,k,1],strides=[1,k,k,1])
    # ksize is the window size , it determines by how much we have to cover in every stride ,we cover k*k*1 every stride (depth is taken to be 1 as we don't move along the depth i.e we keep the depth to be fixed)
    # strides tells us by what stride we have to move everytime , which is same as k 

In [10]:
x = tf.placeholder('float',[None,input_pixels])
y = tf.placeholder(tf.int32,[None,n_out])
kprob = tf.placeholder('float')
pred = cnn(x,weights,biases,kprob)

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred,labels=y))

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimize = optimizer.minimize(cost)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(5, 5, 1, 32) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(5, 5, 32, 64) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(3136, 1024) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=(1024, 10) dtype=float32_ref>,
 <tf.Variable 'Variable_4:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'Variable_5:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'Variable_6:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'Variable_7:0' shape=(10,) dtype=float32_ref>]

In [16]:
batch_size=100
for i in range(10):
    num_batches=int(mnist.train.num_examples/batch_size)
    total_cost = 0
    for j in range(num_batches):
        batch_x,batch_y = mnist.train.next_batch(batch_size)
        c,_=sess.run([cost,optimize],feed_dict={x:batch_x,y:batch_y,kprob:0.8}) # keep_prob is for having a dropout layer
        total_cost+=c
    print(total_cost) # total cost seems to be higher at each point ,since we are summing multiple costs

1265076.611101389
36099.417926728725
19281.34519947216
13858.519828341156
10277.11192503862
8573.849303381518
8045.883767451409
7840.456589431392
6291.549566016882
5943.174045584639


In [18]:
# Find the maximum in y for each data point , because that is the digit that I am going to predict
predictions = tf.arg_max(pred,1)
correct_labels = tf.arg_max(y,1)
correct_predictions = tf.equal(predictions,correct_labels)
predictions_evaluated,correct_pred = sess.run([predictions,correct_predictions],feed_dict={x:mnist.test.images,y:mnist.test.labels,kprob:1})
# feed_dict only need x as predictions depend only upon x and not on any other parameter
predictions_evaluated,correct_pred # False mean wrong and True means right
print("Accuracy is :" ,correct_pred.sum()/len(mnist.test.labels) * 100,"%")

Accuracy is : 98.03 %
